
<a href="https://colab.research.google.com/github/kokchun/Machine-learning-AI22/blob/main/Exercises/E00_linear_regression.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; to see hints and answers.

---
# Linear regression exercises

---
These are introductory exercises in Machine learning with focus in **linear regression** .

<p class = "alert alert-info" role="alert"><b>Note</b> all datasets used in this exercise can be found under Data folder of the course Github repo</p>

<p class = "alert alert-info" role="alert"><b>Note</b> that in cases when you start to repeat code, try not to. Create functions to reuse code instead. </p>

<p class = "alert alert-info" role="alert"><b>Remember</b> to use <b>descriptive variable, function, index </b> and <b> column names</b> in order to get readable code </p>

The number of stars (\*), (\*\*), (\*\*\*) denotes the difficulty level of the task

---

## 0. Simulate phone dataset (*)

We want to simulate data $(x,y)$ to represent cost for phone subscriptions, with: 

- $x$ - called minutes per month
- $y$ - SEK per month 

&nbsp; a) Use ```numpy.random.normal()``` to simulate a dataset with the following requirements:(*)
- set a seed to 42 (for reproducibility and reference)
- simulate 400 x-values from the r.v. $X \sim \mathcal{N}(100, 100)$ 
- take absolute value of these x-values
- simulate noise 400 noise values from r.v. $\epsilon \sim \mathcal{N(0, 50)}$ 
- Let $y = 2x+25+\epsilon$
- plot the data set 

&nbsp; b) Now we want to remove some outliers according to this assumption: (*)
- no one talks more than 300 min using this type of subscription
- no ones costs can be negative
- plot the new dataset
- also plot ground truth using the true parameters $\beta_0 = 25, \beta_1 = 2$

&nbsp; c) Insert the values into a DataFrame (*)

<details>

<summary>Answer</summary>

<img src="../assets/simulated_phone_dataset_0.png" height="200"/>

a) 

Number of points x ≥ 300 min: 8

Number of points y < 0 kr: 6


b)

Length of x, outliers removed 386

Length of y, outliers removed 386

c)

df.head()

|    |   Minutes |     Cost |
|---:|----------:|---------:|
|  0 |   59.4428 | 168.721  |
|  1 |   40.0625 |  98.2118 |
|  2 |  100.524  | 258.433  |
|  3 |  104.698  | 310.548  |
|  4 |   54.9935 | 123.279  |


</details>

---

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# random seed for reproducibility
np.random.seed(42)

# 400 random normally distributed points with mu = 100, sigma = 100
X = abs(np.random.normal(100, 100, 400))
print(f"X:\n{X[:4]}\n{max(X) = }\n{min(X) = }\n")

# 400 random normally distributed noise points with mu = 0, sigma = 50
epsilon = np.random.normal(0, 50, 400)
print(f"epsilon:\n{epsilon[:4]}\n{max(epsilon) = }\n{min(epsilon) = }\n")

beta_0 = 25 # intercept
beta_1 = 2  # slope

# y as a function of X with random noise added
y = beta_0 + beta_1 * X + epsilon

print(f"y:\n{y[:4]}\n{max(y) = }\n{min(y) = }\n")

In [ ]:
sns.scatterplot(x = X, y = y, label = "Datapoints")
plt.axhline(y = 0, color = "r", linestyle = "--", label = "Min Cost Cutoff")
plt.axvline(x = 300, color = "g", linestyle = "-.", label = "Max Time Cutoff")

plt.legend()

In [ ]:
#B

# getting indices where time is above 300
itemindex = np.where(X > 300)

# times above 300
X[itemindex]

In [ ]:
# deleting indices of X outliers from both X and y
X = np.delete(X, itemindex)
y = np.delete(y, itemindex)

In [ ]:
# getting indices where cost is below 0
itemindex = np.where(y < 0)

# costs below 0
y[itemindex]

In [ ]:
# deleting indices of y outliers from both X and y
X = np.delete(X, itemindex)
y = np.delete(y, itemindex)

In [ ]:
# no more values below 0 cost or above 300 time
np.where(X > 300), np.where(y < 0)

In [ ]:
sns.regplot(x = X, y = y, label = "Datapoints", line_kws={"color": "black", "label": "Based on Entire Dataset"})
sns.lineplot(x = X, y = 2 * X + 25, color = "red", label = "OLS Prediction")

plt.legend();

In [ ]:
#C
df = pd.DataFrame()

df["X"] = X
df["y"] = y

df.head()

## 1. Train|test split (*)

Before moving on with linear regression we shall first perform a train-test-split. 

&nbsp; a) Create a train-test-split function with the following call signature: (*)

```py
def train_test_split(X: pd.DataFrame, y: pd.DataFrame, train_fraction=.7: float, random_state=42: int, replace=False: bool) -> tuple
```

that returns the tuple:
```
(X_train, X_test, y_train, y_test)
```

&nbsp; b) Now use this to split up your data into a training set and test set. Check manually that the split is performed correctly. (*)


<details>

<summary>Hint</summary>

b) Check the length of each set, and check the indices of the sorted sets that they don't overlap and are not missing. Also check that they sum up to what you expect.

</details>

<br/>

<details>

<summary>Answer</summary>

Using default 0.7: 
- length of X_train: 270
- length of X_test: 116
- length of y_train: 270
- length of X_test: 116

</details>

---

In [ ]:
def test_train_split(X, y, train_fraction=0.7, random_state=42, replace=False) -> tuple:
    
    # Set seed for reproducibility
    np.random.seed(random_state)

    # Calculate the number of indices for training data
    n = int(len(X) * train_fraction)

    # Generate an array of random indices
    indices = np.random.choice(len(X), size=n, replace=replace)

    # Split X and y into train and test data based on the indices array
    X_train = X[indices]
    X_test = X[np.in1d(np.arange(len(X)), indices, invert=True)]
    y_train = y[indices]
    y_test = y[np.in1d(np.arange(len(y)), indices, invert=True)]

    return X_train, X_test, y_train, y_test

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = test_test_test(X, y)

# checking dimensions to make sure everything looks right
X_train.shape, y_train.shape, X_test.shape, y_test.shape

## 2. Simple linear regression with normal equation (*)

Use the normal equation for simple linear regression to solve for the coefficients $\hat{\beta} = (\beta_0, \beta_1)$. Note that you should only use the training data to fit the regression line, and not data from the test set. Plot the the regression line, together with ground truth and training data. 

<details>

<summary>Hint</summary>

It is important to keep track of the shapes of the vectors, matrices in order for matrix multiplication matmul "@" to work correctly. Also, if you have series object, you need to convert it to numpy. 

</details>

<br/>

<details>

<summary>Answer</summary>


<img src="../assets/Reg_line_normal_eq.png" height="200"/>

</details>

---

In [ ]:
# adding a column of ones for coming calculation
ones = np.ones((len(X_train),))
X_train_matrix = np.stack((ones, X_train), axis= -1)

X_train_matrix[:3], X_train_matrix.shape

In [ ]:
# adding a column of ones for coming calculation
ones = np.ones((len(X_test),))
X_test_matrix = np.stack((ones, X_test), axis= -1)

X_test_matrix[:3], X_test_matrix.shape

In [ ]:
# defining function for estimating unknown parameters (beta vector)
OLS = lambda X, y: np.linalg.inv(X.T @ X) @ X.T @ y

# using function to estimate beta_hat based on training data
beta_hat = OLS(X_train_matrix, y_train)

beta_hat

In [ ]:
# looking at dimensions to make sure everything is ready to move on
beta_hat.shape, X_test.shape, y_test.shape

## 3. Prediction and evaluation (*)

&nbsp; a) Use your model to make prediction on testing data. Plot the prediction cost against X_test, and y_test against X_test. (*)

&nbsp; b) Calculate MAE, MSE, RMSE (*)

<details>

<summary>Hint</summary>

Calculate y_pred from X_test and use y_test and y_pred to compute different evaluation metrics.

Careful with dimensions when computing the evaluation metrics, else it can be catastrophical logical errors due to numpy broadcasting feature.

Note that after you have calculate the error metrics on test data you are not allowed to change any parameters to make the line fit better to the testing data.

</details>

<br/>

<details>

<summary>Answer</summary>

a) 

<img src="../assets/eval_simple_lin_reg.png" height="200"/>

b)

Mean absolute error on testing data: 36.97 kr

Mean squared error on testing data: 2374 kr^2

Root mean squared error on testing data: 48.72 kr

</details>

---

In [ ]:
# prediction function
predict = lambda x, beta: np.dot(x, beta)

# storing predictions to y_pred, reshaping from (115,1) to (115,)
y_pred = predict(X_test_matrix, beta_hat)

y_pred[:3], y_pred.shape

In [ ]:
sns.scatterplot(x = X_test, y = y_test, color = "blue", alpha = 0.6, label = "True Values")
sns.scatterplot(x = X_test, y = y_pred, color = "red", alpha = 0.6, label = "Predicted Values")
sns.lineplot(x = X_test, y = y_pred, color = "black", zorder = 0, label = "Regression Line");

In [ ]:
#B
m = len(y_test)

mean_absolute_error = np.sum(np.abs(y_test - y_pred)) / m

mean_squared_error = np.sum((y_test - y_pred) ** 2) / m

root_mean_squared_error = np.sqrt(mean_squared_error)


print(f"MAE: {mean_absolute_error}")
print(f"MSE: {mean_squared_error}")
print(f"RMSE: {root_mean_squared_error}")

## 4. Simulate more explanatory variables (\*)

Now we will simulate the explanatory variables for minutes, text messages and amount of surf. For reference and reproducibility use numpy random seed 42. Assume there is:

- mean start cost: 25kr
- mean cost per minute: 2kr
- mean cost per sms: 50 öre
- mean cost per GB: 50kr

Then the model for the cost will be:
$y = 25 + 2x_1 + 0.5x_2 + 50x_3 + \epsilon$, where

- $x_i$ sampled from r.v. $X_i$ for $i = \{1,2,3\}$
- $X_1 \sim |\mathcal{N}(100,100)|$, (absolute value)
- $X_2 \sim \mathcal{U}(0,50)$, (discrete uniform distribution)
- $X_3 \sim |\mathcal{N}(0,2)|$,
- $\epsilon \sim \mathcal{N}(0,50)$

&nbsp; a) Simulate 10000 samples of each of $x_1, x_2, x_3$ and $y$ and save them in a DataFrame. Also add an intercept column containing ones. (\*)

&nbsp; b) Make histograms for each of the explanatory variables $x_1, x_2, x_3$ and the response variable $y$ (\*)

&nbsp; c) Clean the data using the following constraints (\*)

- surf should be less than 4
- minutes should be less than 300
- cost should be larger than 0

&nbsp; d) Make new histograms for the variables. (\*)

<details>

<summary>Hint</summary>
Your data analysis skill toolbox together with statistics and linear algebra skills are getting quite handy here.

</details>

<br/>

<details>

<summary>Answer</summary>

a)

|      | Intercept | Minutes | SMS | Surf (GB) |    Cost |
| ---: | --------: | ------: | --: | --------: | ------: |
|    0 |         1 | 149.671 |  41 |   2.26301 | 502.396 |
|    1 |         1 | 86.1736 |  16 | 0.0315695 | 179.072 |
|  ... |       ... |     ... | ... |       ... |     ... |
| 9318 |         1 | 149.577 |  31 |   3.43929 | 536.176 |
| 9319 |         1 | 164.439 |  43 |   1.40641 | 406.674 |

b)

<img src="../assets/hist_variables.png" height="200"/>

d)

<img src="../assets/hist_var_cleaned.png" height="200"/>

</details>

---


In [ ]:
def simulate_sample(sample_size = 10000):

    np.random.seed(42) # for reproducability

    x1 = np.abs(np.random.normal(loc = 100, scale = 100, size = sample_size))
    x2 = np.random.uniform(low = 0, high = 50, size = sample_size)
    x3 = np.abs(np.random.normal(loc = 0, scale = 2, size = sample_size))
    epsilon = np.random.normal(loc = 0, scale = 50, size = sample_size)

    beta_0 = 25
    beta_1 = 2
    beta_2 = 0.5
    beta_3 = 50

    y = beta_0 + beta_1 * x1 + beta_2 * x2 + beta_3 * x3 + epsilon

    return x1, x2, x3, y

In [ ]:
x1, x2, x3, y = simulate_sample()

print(f"y:\n{y[:3]}\n{y.shape}")

In [ ]:
#B,
fig, ax = plt.subplots(1, 3, figsize = (12, 4))
variables = ["Minute", "Text Message", "GB"]

for i, x in enumerate([x1, x2, x3]):
    sns.histplot(x = x, y = y, ax = ax[i])
    ax[i].set_title(f"Cost per {variables[i]}")
    ax[i].set_xlabel(variables[i] + "s")

ax[0].set_ylabel("Cost");

In [ ]:
#C
def clean_data(x1, x2, x3, y):
    # minutes must be less than 300
    itemindex = np.where(x1 >= 300)

    x1 = np.delete(x1, itemindex)
    x2 = np.delete(x2, itemindex)
    x3 = np.delete(x3, itemindex)
    y = np.delete(y, itemindex)

    # GB must be less than 4
    itemindex = np.where(x3 >= 4)

    x1 = np.delete(x1, itemindex)
    x2 = np.delete(x2, itemindex)
    x3 = np.delete(x3, itemindex)
    y = np.delete(y, itemindex)

    # cost must be greater than 0
    itemindex = np.where(y <= 0)

    x1 = np.delete(x1, itemindex)
    x2 = np.delete(x2, itemindex)
    x3 = np.delete(x3, itemindex)
    y = np.delete(y, itemindex)

    return x1, x2, x3, y

In [ ]:
#D
fig, ax = plt.subplots(1, 3, figsize = (12, 4))
variables = ["Minute", "Text Message", "GB"]

for i, x in enumerate([x1, x2, x3]):
    sns.histplot(x = x, y = y, ax = ax[i])
    ax[i].set_title(f"Cost per {variables[i]}")
    ax[i].set_xlabel(variables[i] + "s")

ax[0].set_ylabel("Cost");

## 5. Multiple linear regression (*)

&nbsp; a) Perform a train|test split with 0.8 of the data for training. (*)

&nbsp; b) Use the normal equation to compute $\hat{\beta}$ (*)

&nbsp; c) Predict on the test data and compute MAE, MSE and RMSE. (*)

&nbsp; d) Now repeat 4a), 4c), 5a), 5b) using 10, 100, 1000, 10000, 100000, 1000000 samples, and calculate RMSE for each of these simulations. Plot the RMSE against sample size. (**)


<details>

<summary>Hint</summary>

It is important to keep track of the shapes of the vectors, matrices in order for matrix multiplication matmul "@" to work correctly. Also, if you have series object, you need to convert it to numpy. 

</details>

<br/>

<details>
<summary>Answer</summary>


<img src="../assets/RMSE_simulation.png" height="200"/>

</details>

In [ ]:
def insert_ones():
    # adding a column of ones for coming calculation
    ones = np.ones((len(x1),))
    X = np.stack((ones, x1, x2, x3), axis= -1)

    return X

In [ ]:
X = insert_ones()

X[:3], X.shape

In [ ]:
X_train, X_test, y_train, y_test = test_test_test(X, y, train_fraction = 0.8)

# checking dimensions to make sure everything looks right
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
#B
# looking at dimensions to make sure everything is ready to move on
beta_hat.shape, X_test.shape, y_test.shape

In [ ]:
#C
y_pred = predict(X_test, beta_hat)

y_pred[:3], y_pred.shape

In [ ]:
# defining functions to calculate MAE, MSE, RMSE
mean_absolute_error = lambda y_test, y_pred: np.sum(np.abs(y_test - y_pred)) / len(y_test)
mean_squared_error = lambda y_test, y_pred: np.sum((y_test - y_pred) ** 2) / len(y_test)
root_mean_squared_error = lambda y_test, y_pred: np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mean_absolute_error(y_test, y_pred)}")
print(f"MAE: {mean_squared_error(y_test, y_pred)}")
print(f"MAE: {root_mean_squared_error(y_test, y_pred)}")

In [ ]:
#D
samples = [10, 100, 1000, 10000, 100000, 1000000]
RMSE_list = []

for i in samples:
    x1, x2, x3, y = simulate_sample(sample_size = i)
    x1, x2, x3, y = clean_data(x1, x2, x3, y)
    X = insert_ones()
    X_train, X_test, y_train, y_test = test_test_test(X, y, train_fraction = 0.8)
    beta_hat = OLS(X_train, y_train)
    y_pred = predict(X_test, beta_hat)
    RMSE = root_mean_squared_error(y_test, y_pred)
    RMSE_list.append(RMSE)
    
sns.lineplot(x = samples, y = RMSE_list)
plt.suptitle("RMSE as Sample Size Increases")
plt.ylabel("RMSE")
plt.xlabel("Sample Size")
plt.xscale("log")

---

Kokchun Giang

[LinkedIn][linkedIn_kokchun]

[GitHub portfolio][github_portfolio]

[linkedIn_kokchun]: https://www.linkedin.com/in/kokchungiang/
[github_portfolio]: https://github.com/kokchun/Portfolio-Kokchun-Giang

---